In [1]:
from PIL import Image
import requests

from transformers import CLIPProcessor, CLIPModel

/home/ai02/miniconda3/envs/kk/lib/python3.8/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/ai02/miniconda3/envs/kk/lib/python3.8/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/ai02/miniconda3/envs/kk/lib/python3.8/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [15]:
model = CLIPModel.from_pretrained('/home/ai02/Desktop/clip-vit-base-patch32')
processor = CLIPProcessor.from_pretrained('/home/ai02/Desktop/clip-vit-base-patch32')
image = Image.open('/media/ai02/43633275BF4935F0/kk/datasets/RUOD/images/test/003926.jpg')

In [16]:
text = ['God', 'turtle', 'bus']
inputs = processor(text=text, images=image, return_tensors='pt', padding=True)

output = model(**inputs)
logits = output.logits_per_image

In [17]:
print(logits.softmax(dim=1))

tensor([[2.1311e-04, 9.9976e-01, 2.5429e-05]], grad_fn=<SoftmaxBackward0>)


In [2]:
!git clone https://github.com/openai/CLIP.git

Cloning into 'CLIP'...
remote: Enumerating objects: 251, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 251 (delta 3), reused 3 (delta 0), pack-reused 243 objects:  34% (87/251), 2.04 MiB | 1016.00 KiB/sReceiving objects:  34% (87/251), 2.96 MiB | 740.00 KiB/sReceiving objects:  34% (87/251), 3.34 MiB | 378.00 KiB/sReceiving objects:  34% (87/251), 3.50 MiB | 189.00 KiB/sReceiving objects:  34% (87/251), 3.71 MiB | 116.00 KiB/sReceiving objects:  34% (87/251), 3.99 MiB | 114.00 KiB/sReceiving objects:  35% (88/251), 4.15 MiB | 135.00 KiB/sReceiving objects:  45% (114/251), 4.30 MiB | 145.00 KiB/sReceiving objects:  45% (114/251), 4.58 MiB | 146.00 KiB/sReceiving objects:  45% (114/251), 4.77 MiB | 111.00 KiB/sReceiving objects:  45% (114/251), 4.93 MiB | 104.00 KiB/sReceiving objects:  45% (114/251), 5.28 MiB | 129.00 KiB/sReceiving objects:  45% (114/251), 5.62 MiB | 158.00 KiB/sReceiving objects:  45% (114/251), 5.96 MiB

In [4]:
!pip install CLIP

  Preparing metadata (setup.py) ... done
  Created wheel for CLIP: filename=clip-0.2.0-py3-none-any.whl size=6988 sha256=bd63c8fece2b6beb7741847d8b80fb63fde0241c4dfed25c5aa287f04521e19d
  Stored in directory: /home/ai02/.cache/pip/wheels/6d/17/18/6193c6b02f9e35e3b3f0721a349b9f9f74bac11feb0f86fdd1
Successfully built CLIP
DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [12]:
from CLIP import clip
import torch

text = ['people']
model, _ = clip.load("ViT-B/32")
device = next(model.parameters()).device
text_token = clip.tokenize(text).to(device)
txt_feats = model.encode_text(text_token).to(dtype=torch.float32)
txt_feats = txt_feats / txt_feats.norm(p=2, dim=-1, keepdim=True)
txt_feats = txt_feats.reshape(-1, len(text), txt_feats.shape[-1]).detach()
# self.model[-1].nc = len(text)

In [10]:
# txt_feats.shape

torch.Size([1, 512])

In [6]:
import torch
from torch import nn

def autopad(k, p=None, d=1):  # kernel, padding, dilation
    """Pad to 'same' shape outputs."""
    if d > 1:
        k = d * (k - 1) + 1 if isinstance(k, int) else [d * (x - 1) + 1 for x in k]  # actual kernel-size
    if p is None:
        p = k // 2 if isinstance(k, int) else [x // 2 for x in k]  # auto-pad
    return p


class Conv(nn.Module):
    """Standard convolution with args(ch_in, ch_out, kernel, stride, padding, groups, dilation, activation)."""

    default_act = nn.SiLU()  # default activation

    def __init__(self, c1, c2, k=1, s=1, p=None, g=1, d=1, act=True):
        """Initialize Conv layer with given arguments including activation."""
        super().__init__()
        self.conv = nn.Conv2d(c1, c2, k, s, autopad(k, p, d), groups=g, dilation=d, bias=False)
        self.bn = nn.BatchNorm2d(c2)
        self.act = self.default_act if act is True else act if isinstance(act, nn.Module) else nn.Identity()

    def forward(self, x):
        """Apply convolution, batch normalization and activation to input tensor."""
        return self.act(self.bn(self.conv(x)))

    def forward_fuse(self, x):
        """Perform transposed convolution of 2D data."""
        return self.act(self.conv(x))


class MaxSigmoidAttnBlock(nn.Module):
    """Max Sigmoid attention block."""

    def __init__(self, c1, c2, nh=1, ec=128, gc=512, scale=False):
        """
        Initializes MaxSigmoidAttnBlock with specified arguments.
        :param nh: Number of heads
        :param hc: Number of head channels
        :param
        """
        super().__init__()
        self.nh = nh
        self.hc = c2 // nh
        self.ec = Conv(c1, ec, k=1, act=False) if c1 != ec else None
        self.gl = nn.Linear(gc, ec)
        self.bias = nn.Parameter(torch.zeros(nh))
        self.proj_conv = Conv(c1, c2, k=3, s=1, act=False)
        self.scale = nn.Parameter(torch.ones(1, nh, 1, 1)) if scale else 1.0

    def forward(self, x, guide):
        """
        Forward process.
        :param x: Input
        :param guide: text_features [bs, len(nc), embeddings]

        """
        bs, _, h, w = x.shape

        guide = self.gl(guide)
        guide = guide.view(bs, -1, self.nh, self.hc)
        embed = self.ec(x) if self.ec is not None else x
        embed = embed.view(bs, self.nh, self.hc, h, w)

        aw = torch.einsum("bmchw,bnmc->bmhwn", embed, guide)
        aw = aw.max(dim=-1)[0]
        aw = aw / (self.hc**0.5)
        aw = aw + self.bias[None, :, None, None]
        aw = aw.sigmoid() * self.scale

        x = self.proj_conv(x)
        x = x.view(bs, self.nh, -1, h, w)
        x = x * aw.unsqueeze(2)
        return x.view(bs, -1, h, w)
    
model = MaxSigmoidAttnBlock(256, 256, 8, 256)
x = torch.randn(16, 256, 40, 40)
guide = torch.randn(16, 80, 512)
y = model(x, guide)

KeyboardInterrupt: 